# 02: Generate gridded WBGT in the shade estimates
*Use downscaled CMIP6 projections from the [NEX-GDDP-CMIP6 dataset](https://www.nccs.nasa.gov/services/data-collections/land-based-products/nex-gddp-cmip6) to generate gridded estimates of WBGT. The projections cover historical and future (SSP2-4.5) periods at a daily timestep and 0.25 degree resolution for the entire globe's land surface.*

In [1]:
import os

import coiled
import dask
import numpy as np
import thermofeel as tf
import xarray as xr
import xclim

# from tqdm import tqdm
from utils import gcm_list, load_virtual_nasa_nex, wbgt

os.environ["USE_PYGEOS"] = "0"

In [ ]:
cluster = coiled.Cluster(
    n_workers=2,
    name="02",
    worker_vm_types=["m7g.medium"],
    scheduler_vm_types=["c7g.large"],
    region="us-west-2",
    spot_policy="spot_with_fallback",
)

cluster.adapt(minimum=1, maximum=200)

client = cluster.get_client()

In [2]:
def adjust_pressure(temperature, elevation):
    """
    Approximate surface pressure given the elevation and temperature.
    Method from https://doi.org/10.1038/s41598-019-50047-w
    """
    return 101325 * np.power(10, -elevation / (18400 * temperature / 273.15))

Set up cluster to handle multiprocessing using a Dask client.

Read in elevation data, which was processed in `01_elevation.ipynb`.

In [3]:
elev = xr.open_zarr(
    "s3://carbonplan-climate-impacts/extreme-heat/v1.0/inputs/elevation.zarr"
)
elev = elev.chunk({"lat": -1, "lon": -1}).compute()

Identify which scenarios and years to evaluate.

In [7]:
scenarios = ["historical", "ssp245", "ssp370"]


def build_parameters() -> list:

    param_list = []
    for gcm in gcm_list:
        for scenario in scenarios:
            param_list.append((gcm, scenario))
    return param_list


param_tuples = build_parameters()

Calculate future projections of WBGT.


In [4]:
def process(param_tuple: tuple):

    elev = xr.open_zarr(
        "s3://carbonplan-climate-impacts/extreme-heat/v1.0/inputs/elevation.zarr"
    )
    elev = elev.chunk({"lat": -1, "lon": -1}).compute()

    variables = ["tasmax", "huss", "tas"]

    gcm, scenario = param_tuple

    id_string = f"{gcm}-{scenario}"
    #####################
    # NOTE TEMP UPDATED!
    #####################

    output = (
        f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-"
        f"gridded/years/{gcm}/{id_string}_impacts-impacts-extreme-heat-test-pinned_env.zarr"
    )
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!
    #### NOTE: WARNING SUBSET!!!
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!

    ds = load_virtual_nasa_nex(gcm=gcm, scenario=scenario).sel(time=slice("1985-01-01","1985-12-31"))[variables]
    ds = ds.chunk({"time": 15})

    # # calculate elevation-adjusted pressure
    ds["ps"] = xr.apply_ufunc(adjust_pressure, ds["tas"], elev, dask="allowed").rename(
        {"elevation": "ps"}
    )["ps"]
    ds["ps"].attrs["units"] = "Pa"
    ds["hurs"] = xclim.indices.relative_humidity(
        tas=ds["tasmax"], huss=ds["huss"], ps=ds["ps"]
    )
    ds["tasmax"].attrs = {}

    # windspeed assumption of 0.5 m/s (approximating shaded/indoor
    # conditions)
    ds["sfcWind"] = (ds["tas"] - ds["tas"]) + 0.5
    ds["WBT"] = tf.thermofeel.calculate_wbt(ds["tasmax"] - 273.15, ds["hurs"])

    ds["BGT"] = tf.thermofeel.calculate_bgt(ds["tasmax"], ds["tasmax"], ds["sfcWind"])
    ds["WBGT"] = wbgt(ds["WBT"], ds["BGT"], ds["tasmax"] - 273.15)
    ds["WBGT"].attrs["units"] = "degC"
    ds = ds[["WBGT"]]
    ds = dask.optimize(ds)[0]
    t = ds.to_zarr(output, consolidated=True, mode="w", compute=False)
    t.compute()
    return output

In [ ]:
for input_params in param_tuples[0:1]:
    process(param_tuple=input_params)

In [ ]:
cluster.shutdown()

In [ ]:
# gcm, scenario = param_tuples[0]
# id_string = f"{gcm}-{scenario}"

# output = f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-gridded/years/{gcm}/{id_string}_impacts-impacts-extreme-heat-test-env.zarr"
# output

# new = xr.open_zarr(output, chunks={})

In [ ]:
# f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-gridded/years/{gcm}/{id_string}_impacts-impacts-extreme-heat-test-env.zarr"

In [ ]:
# old = xr.open_zarr(
#     f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-gridded/years/{gcm}/{id_string}.zarr",
#     chunks={},
# )

In [ ]:
# f"s3://carbonplan-scratch/extreme-heat/wbgt-shade-gridded/years/{gcm}/{id_string}.zarr"

In [ ]:
new.WBGT.isel(time=slice(0, 100)).sel(lat=40, lon=40, method="nearest").plot(x="time")
old.WBGT.isel(time=slice(0, 100)).sel(lat=40, lon=40, method="nearest").plot(x="time")

In [ ]:
cluster.shutdown()

In [2]:
import xarray as xr 
nebari_ori = xr.open_zarr('s3://carbonplan-scratch/extreme-heat-extension/wbgt-shade-gridded/years/ACCESS-CM2/ACCESS-CM2-historical-1985-ori.zarr')
nicest_code = xr.open_zarr('s3://carbonplan-scratch/extreme-heat/wbgt-shade-gridded/years/ACCESS-CM2/ACCESS-CM2-historical_impacts-impacts-extreme-heat-test-pinned_env.zarr')



In [ ]:
nebari_ori = xr.open_zarr('s3://carbonplan-scratch/extreme-heat-extension/wbgt-shade-gridded/years/ACCESS-CM2/ACCESS-CM2-historical-1985-ori.zarr')

nicest_code = xr.open_zarr('s3://carbonplan-scratch/extreme-heat/wbgt-shade-gridded/years/ACCESS-CM2/ACCESS-CM2-historical_impacts-impacts-extreme-heat-test-pinned_env.zarr')
# (nebari_ori.sel(time=slice("1985-01-01","1985-12-31")).isel(time=0)-nicest_code.sel(time=slice("1985-01-01","1985-12-31")).isel(time=0)).WBGT.plot()

In [ ]:
nebari_ori.isel(time=0).time

In [ ]:
nicest_code.isel(time=0).WBGT.values == nebari_ori.isel(time=0).WBGT.values